<a href="https://colab.research.google.com/github/Setstock/Homework/blob/main/LSTM_Im30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.sparse import csr_matrix

%config InlineBackend.figure_format = 'retina'

# Import dataset
df = pd.read_csv('/content/30-datamodels.csv')
processed_df = pd.DataFrame(df)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
scaler = StandardScaler()
temp_list=['ราคาสูงสุด','ราคาต่ำสุด','volume','A/B','P/R']
processed_df[temp_list] = scaler.fit_transform(processed_df[temp_list])

In [3]:
# Remove commas and convert to float
processed_df[temp_list] = processed_df[temp_list].replace({',': ''}, regex=True).astype(float)

# Now you can apply the scaler
scaler = StandardScaler()
processed_df[temp_list] = scaler.fit_transform(processed_df[temp_list])


In [4]:
print(processed_df[temp_list])

     ราคาสูงสุด  ราคาต่ำสุด    volume       A/B       P/R
0      0.264647    0.271179  0.548389 -0.469433  0.244257
1      0.264647    0.369161  0.402965 -0.396465  0.235792
2      0.272765    0.305472  0.440262 -0.399755  0.238276
3      0.199703    0.251582  0.423420 -0.434965  0.283733
4      0.236234    0.261380  0.452912 -0.399920  0.264685
..          ...         ...       ...       ...       ...
743   -0.112839   -0.077638 -0.421045 -0.477495  0.194752
744   -0.060072   -0.075678 -0.383548 -0.467459  0.138621
745   -0.076308   -0.042364 -0.397863 -0.480950  0.197420
746    0.012990   -0.042364 -0.402105 -0.508180  0.060959
747    0.102287    0.055618 -0.167474 -0.457916  0.158681

[748 rows x 5 columns]


In [5]:
processed_df

,ราคาสูงสุด,ราคาต่ำสุด,volume,A/B,P/R,label
0,0.264647,0.271179,0.548389,-0.469433,0.244257,0
1,0.264647,0.369161,0.402965,-0.396465,0.235792,0
2,0.272765,0.305472,0.440262,-0.399755,0.238276,0
3,0.199703,0.251582,0.423420,-0.434965,0.283733,0
4,0.236234,0.261380,0.452912,-0.399920,0.264685,0
...,...,...,...,...,...,...
743,-0.112839,-0.077638,-0.421045,-0.477495,0.194752,0
744,-0.060072,-0.075678,-0.383548,-0.467459,0.138621,0
745,-0.076308,-0.042364,-0.397863,-0.480950,0.197420,0
746,0.012990,-0.042364,-0.402105,-0.508180,0.060959,1


In [8]:
  #do not run BC df has not column for delet

# Drop the rightmost (last) column
#processed_df = processed_df.drop(processed_df.columns[-3], axis=1)

In [9]:
    #Do not run BC df not change

#processed_df

Display All rows.

In [10]:
# Set the option to display all rows
#pd.set_option('display.max_rows', None)

# Display the DataFrame
#display(processed_df)            print

In [11]:
# Find Max Min all DataFrame
#overall_max_value = processed_df.max().max()
#overall_min_value = processed_df.min().min()

# out put
#print(f"\nค่าสูงสุดทั้งหมด: {overall_max_value}")
#print(f"ค่าต่ำสุดทั้งหมด: {overall_min_value}")


find Max Min  data has more difference
Max 15   Min -21.99

In [6]:
import numpy as np
import pandas as pd

# Ensure columns contain numeric data
for column in processed_df.columns[0:5]:
    processed_df[column] = pd.to_numeric(processed_df[column], errors='coerce')

# Iterate through the columns from index 0 to 5
for column in processed_df.columns[0:5]:
    # Calculate quartiles for the current column
    quartiles = processed_df[column].quantile([0.25, 0.5, 0.75])

    # Iterate through the rows in the current column
    for index, value in processed_df[column].items():
        if pd.isna(value):
            continue  # Skip NaN values
        if 0.0 <= value <= quartiles[0.25]:
            processed_df.at[index, column] = "A"
        elif quartiles[0.25] < value <= quartiles[0.5]:
            processed_df.at[index, column] = "B"
        elif quartiles[0.5] < value <= quartiles[0.75]:
            processed_df.at[index, column] = "C"
        else:
            processed_df.at[index, column] = "D"

<ipython-input-6-5b508957d033>:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'D' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  processed_df.at[index, column] = "D"
<ipython-input-6-5b508957d033>:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'D' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  processed_df.at[index, column] = "D"
<ipython-input-6-5b508957d033>:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'D' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  processed_df.at[index, column] = "D"
<ipython-input-6-5b508957d033>:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Va

In [7]:
# Concatenate the columns into a new 'gencode' column
processed_df['gencode'] = processed_df.apply(
    lambda row: ''.join(row[['ราคาสูงสุด', 'ราคาต่ำสุด', 'volume', 'A/B', 'P/R']]), axis=1)

# Display the DataFrame
print(processed_df[['gencode', 'label']])

    gencode  label
0     DDDBD      0
1     DDDBD      0
2     DDDBD      0
3     DDDBD      0
4     DDDBD      0
..      ...    ...
743   CCBBC      0
744   CCBBC      0
745   CCBBC      0
746   DCBBB      1
747   DDCBC      1

[748 rows x 2 columns]


In [8]:
processed_df

,ราคาสูงสุด,ราคาต่ำสุด,volume,A/B,P/R,label,gencode
0,D,D,D,B,D,0,DDDBD
1,D,D,D,B,D,0,DDDBD
2,D,D,D,B,D,0,DDDBD
3,D,D,D,B,D,0,DDDBD
4,D,D,D,B,D,0,DDDBD
...,...,...,...,...,...,...,...
743,C,C,B,B,C,0,CCBBC
744,C,C,B,B,C,0,CCBBC
745,C,C,B,B,C,0,CCBBC
746,D,C,B,B,B,1,DCBBB


In [9]:
from keras.utils import to_categorical

# Assuming df is your DataFrame with the data
text_data = processed_df['gencode'].tolist()
labels = processed_df['label'].tolist()

# Initialize the tokenizer
tokenizer = Tokenizer(char_level=True)

# Fit the tokenizer on the text data
tokenizer.fit_on_texts(text_data)

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(text_data)

# Get the word index mapping
word_index = tokenizer.word_index


# Pad sequences to make them uniform length (optional)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')#xissequence

# Display the results
print("Word Index:", word_index)
print("Sequences:", sequences)
print("Padded Sequences:", padded_sequences)

# One-hot encode the labels
one_hot_labels = to_categorical(labels) #yisonehot

# Display the one-hot encoded labels
print("One-Hot Encoded Labels:")
print(one_hot_labels)



Word Index: {'d': 1, 'c': 2, 'b': 3, 'a': 4}
Sequences: [[1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 2], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 2], [1, 1, 1, 3, 1], [1, 1, 1, 2, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 2, 1], [1, 1, 1, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 2, 1], [1, 1, 1, 2, 1], [1, 1, 3, 1, 1], [1, 1, 2, 1, 1], [1, 1, 3, 3, 1], [1, 1, 2, 3, 1], [1, 1, 1, 3, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 3], [1, 1, 1, 1, 3], [1, 1, 1, 1, 3], [1, 1, 2, 1, 3], [1, 1, 2, 1, 1], [1, 1, 2, 1, 2], [1, 1, 2, 1, 3], [1, 1, 2, 1, 1], [1, 1, 2, 1, 3], [1, 1, 1, 1, 3], [1, 1, 1, 1, 2], [1, 1, 1, 1, 1], [1, 1, 2, 1, 1], [1, 1, 2, 1, 1], [1, 1, 2, 1, 1], [1, 1, 2, 1, 1], [1, 1, 2, 1, 3], [1, 1, 2, 1, 1], [1, 1, 2, 1, 1], [1, 1, 2, 1, 1], [1, 1, 1, 1, 2], [1, 3, 2, 1, 1], [1, 3, 2, 1, 3], [3, 3, 1, 1, 1], [3, 3, 2, 1, 3], [3, 3, 2, 1, 3], [3, 1, 2

In [10]:
X = sequences
y = one_hot_labels

In [11]:
n_splits = len(X) - 4  # Adjust this based on your desired pattern

for i in range(n_splits):
    trainIndex = range(i, i + 4)
    testIndex = i + 4
    #print(f'Train index: {trainIndex}')
    #print(f'Test index: {testIndex}')



In [12]:
#trainIndex = list(range(i, i + 3))
#testIndex = range(i + 3, i + 4)
n_splits = len(X) - 4  # Adjust this based on your desired pattern

for i in range(n_splits):
    trainIndex = list(range(i, i + 4))
    testIndex = range(i + 4, i + 5)
    #print(f'Train index: {trainIndex}')
    #print(f'Test index: {list(testIndex)}')


In [13]:
# Create overlapping train-test pairs

# Specify the window size
window_size = 4

# Create overlapping train-test pairs
for i in range(len(text_data) - window_size):
    train_idx = range(i, i + window_size)
    test_idx = i + window_size

    X_train = padded_sequences[train_idx]
    y_train = one_hot_labels[train_idx]

    X_test = padded_sequences[test_idx]
    y_test = one_hot_labels[test_idx]

In [14]:
y_test

array([0., 1.])

In [15]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
# from tensorflow.keras.optimizers import Adam


# # # Reshape input data for LSTM
# # X_train = padded_sequences[:, None, :]
# # y_train = one_hot_labels

# model = Sequential()
# model.add(LSTM(units=128, input_shape=(None, max_length), return_sequences=True))
# # Increase LSTM units for potentially better representation learning
# model.add(LSTM(units=64))  # Additional LSTM layer
# model.add(Dense(units=2, activation='softmax'))  # Use softmax for multiclass classification

# # Using Adam optimizer with a learning rate of 0.001, which is commonly a good starting point
# optimizer = Adam(learning_rate=0.001)

# # sparse_categorical_crossentropy
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(X_train, y_train, epochs=10, batch_size=32)



Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_1/Cast:0", shape=(None, 5), dtype=float32). Expected shape (None, None, 5), but input has incompatible shape (None, 5)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 5), dtype=int32)
  • training=True
  • mask=None

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import precision_score, recall_score

# Reshape input data for LSTM
X_train = padded_sequences[:, None, :]
y_train = one_hot_labels

# Initialize the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(None, max_length)))  # None indicates variable sequence length
model.add(Dense(units=2, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32)


Epoch 1/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5233 - loss: 0.6897
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6181 - loss: 0.6701  
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6083 - loss: 0.6716 
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6192 - loss: 0.6664
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6112 - loss: 0.6686
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6181 - loss: 0.6649
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6107 - loss: 0.6679
Epoch 8/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5937 - loss: 0.6761
Epoch 9/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6348 - loss: 0.6556
Epoch 10/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6221 - loss: 0.6629
Epoch 11/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6177 - loss: 0.6638
Epoch 12/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6035 -

In [17]:
from sklearn.metrics import precision_score, recall_score

# ...
# Reshape input data for LSTM
X_test_reshaped = padded_sequences[:, None, :]
y_test_one_hot = to_categorical(labels)


# Evaluate the model
test_loss, test_acc = model.evaluate(X_test_reshaped, y_test_one_hot)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

# Make predictions on the test set
y_pred = model.predict(X_test_reshaped)

# Convert predicted probabilities to binary predictions
y_pred_binary = (y_pred > 0.5).astype(int)

# Convert one-hot encoded labels to integers
y_test_int = np.argmax(y_test_one_hot, axis=1)

# Calculate precision and recall
precision = precision_score(y_test_int, y_pred_binary[:, 1])
recall = recall_score(y_test_int, y_pred_binary[:, 1])

print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))


24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5930 - loss: 0.6719   
Test Loss: 0.6659460663795471
Test Accuracy: 0.6122994422912598
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Precision: 0.0
Recall: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


-----No-----